<a href="https://colab.research.google.com/github/VinayWankhede11/Book-Recommendation-System/blob/main/Recommendation_System.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Importing Libraries

In [1]:
import numpy as np
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity

Reading CSV Files

In [2]:
books=pd.read_csv("/content/drive/MyDrive/JTP VIrtual Training/Recommendation System/Books.csv")
books.head(4)

<ipython-input-2-7e99b3d5b992>:1: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  books=pd.read_csv("/content/drive/MyDrive/JTP VIrtual Training/Recommendation System/Books.csv")


,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...
2,0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...


In [3]:
ratings=pd.read_csv("/content/drive/MyDrive/JTP VIrtual Training/Recommendation System/Ratings.csv")
ratings.head(4)

,User-ID,ISBN,Book-Rating
0,276725,034545104X,0
1,276726,0155061224,5
2,276727,0446520802,0
3,276729,052165615X,3


In [4]:
users=pd.read_csv("/content/drive/MyDrive/JTP VIrtual Training/Recommendation System/Users.csv")
users.head(4)

,User-ID,Location,Age
0,1,"nyc, new york, usa",NaN
1,2,"stockton, california, usa",18.0
2,3,"moscow, yukon territory, russia",NaN
3,4,"porto, v.n.gaia, portugal",17.0


Dimensions

In [5]:
print("Books Shape: " ,books.shape )
print("Ratings Shape: " ,ratings.shape )
print("Users Shape: " ,users.shape )

Books Shape:  (271360, 8)
Ratings Shape:  (1149780, 3)
Users Shape:  (278858, 3)


Checking for NULL values

In [6]:
print("Any null values in Books:\n")
print(books.isnull().sum())

Any null values in Books:

ISBN                   0
Book-Title             0
Book-Author            1
Year-Of-Publication    0
Publisher              2
Image-URL-S            0
Image-URL-M            0
Image-URL-L            3
dtype: int64


In [7]:
print("Any null values in Ratings:\n ")
print(ratings.isnull().sum())

Any null values in Ratings:
 
User-ID        0
ISBN           0
Book-Rating    0
dtype: int64


In [8]:
print("Any null values in Users:\n")
print(users.isnull().sum())

Any null values in Users:

User-ID          0
Location         0
Age         110762
dtype: int64


Checking for Duplicate values

In [9]:
print("Any duplicate values in Books:" ,books.duplicated().sum())
print("Any duplicate values in Ratings:",ratings.duplicated().sum())
print("Any duplicate values in Users:",users.duplicated().sum())

Any duplicate values in Books: 0
Any duplicate values in Ratings: 0
Any duplicate values in Users: 0


Removing Missing Values of Age and Ratings

In [10]:
mean_value=users['Age'].mean()
users['Age'].fillna(value=mean_value, inplace=True)

In [11]:
print(users.isnull().sum())

User-ID     0
Location    0
Age         0
dtype: int64


Preprocessing 

In [12]:
ratings_with_book_name=books.merge(ratings,on="ISBN")
ratings_with_book_name.head()

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L,User-ID,Book-Rating
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,2,0
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,8,5
2,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,11400,0
3,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,11676,8
4,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,41385,0


In [13]:
num_zeros = sum(ratings['Book-Rating'] == 0)
print(num_zeros)

716109


In [14]:
df=ratings_with_book_name.copy()
df.dropna(inplace=True)
df.reset_index(drop=True,inplace=True)
df.drop(columns=["ISBN","Year-Of-Publication","Image-URL-S","Image-URL-M"],axis=1,inplace=True)
df.drop(index=df[df["Book-Rating"]==0].index,inplace=True)
# df["Book-Title"]=df["Book-Title"].apply(lambda x: re.sub("[\W_]+"," ",x).strip())
df.head()

,Book-Title,Book-Author,Publisher,Image-URL-L,User-ID,Book-Rating
1,Clara Callan,Richard Bruce Wright,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,8,5
3,Clara Callan,Richard Bruce Wright,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,11676,8
5,Clara Callan,Richard Bruce Wright,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,67544,8
8,Clara Callan,Richard Bruce Wright,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,116866,9
9,Clara Callan,Richard Bruce Wright,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,123629,9


## Popularity Based Recommendation Sysytem
- Popularity based recommendation systems are based on the rating of items by all the users.
- Popularity based recommendation systems works with the trend. It basically uses the items which are in trend right now.

In [15]:
num_of_user_rated_df = ratings_with_book_name.groupby('Book-Title').count()['Book-Rating'].reset_index()
num_of_user_rated_df.rename(columns={'Book-Rating':'Number-of-Users-Rated'},inplace=True)
num_of_user_rated_df

,Book-Title,Number-of-Users-Rated
0,A Light in the Storm: The Civil War Diary of ...,4
1,Always Have Popsicles,1
2,Apple Magic (The Collector's series),1
3,"Ask Lily (Young Women of Faith: Lily Series, ...",1
4,Beyond IBM: Leadership Marketing and Finance ...,1
...,...,...
241066,Ã?Â?lpiraten.,2
241067,Ã?Â?rger mit Produkt X. Roman.,4
241068,Ã?Â?sterlich leben.,1
241069,Ã?Â?stlich der Berge.,3


In [16]:
avg_rating_df = ratings_with_book_name.groupby('Book-Title').mean()['Book-Rating'].reset_index()
avg_rating_df.rename(columns={'Book-Rating':'Average-Ratings'},inplace=True)
avg_rating_df

,Book-Title,Average-Ratings
0,A Light in the Storm: The Civil War Diary of ...,2.250000
1,Always Have Popsicles,0.000000
2,Apple Magic (The Collector's series),0.000000
3,"Ask Lily (Young Women of Faith: Lily Series, ...",8.000000
4,Beyond IBM: Leadership Marketing and Finance ...,0.000000
...,...,...
241066,Ã?Â?lpiraten.,0.000000
241067,Ã?Â?rger mit Produkt X. Roman.,5.250000
241068,Ã?Â?sterlich leben.,7.000000
241069,Ã?Â?stlich der Berge.,2.666667


In [17]:
popular_df = num_of_user_rated_df.merge(avg_rating_df,on='Book-Title')
popular_df

,Book-Title,Number-of-Users-Rated,Average-Ratings
0,A Light in the Storm: The Civil War Diary of ...,4,2.250000
1,Always Have Popsicles,1,0.000000
2,Apple Magic (The Collector's series),1,0.000000
3,"Ask Lily (Young Women of Faith: Lily Series, ...",1,8.000000
4,Beyond IBM: Leadership Marketing and Finance ...,1,0.000000
...,...,...,...
241066,Ã?Â?lpiraten.,2,0.000000
241067,Ã?Â?rger mit Produkt X. Roman.,4,5.250000
241068,Ã?Â?sterlich leben.,1,7.000000
241069,Ã?Â?stlich der Berge.,3,2.666667


In [18]:
def weighted_rate(x):
        v=x["Number-of-Users-Rated"]
        R=x["Average-Ratings"]
        
        return ((v*R) + (m*C)) / (v+m)
    
C=popular_df["Average-Ratings"].mean()
m=popular_df["Number-of-Users-Rated"].quantile(0.90)

In [19]:
popular_df=popular_df[popular_df["Number-of-Users-Rated"] >=250]
popular_df["Popularity"]=popular_df.apply(weighted_rate,axis=1)
popular_df=popular_df.sort_values(by="Popularity",ascending=False)

<ipython-input-19-c67b94b21c0d>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  popular_df["Popularity"]=popular_df.apply(weighted_rate,axis=1)


In [20]:
popular_df

,Book-Title,Number-of-Users-Rated,Average-Ratings,Popularity
80434,Harry Potter and the Prisoner of Azkaban (Book 3),428,5.852804,5.805129
80422,Harry Potter and the Goblet of Fire (Book 4),387,5.824289,5.772160
80441,Harry Potter and the Sorcerer's Stone (Book 1),278,5.737410,5.667477
80426,Harry Potter and the Order of the Phoenix (Boo...,347,5.501441,5.449805
80414,Harry Potter and the Chamber of Secrets (Book 2),556,5.183453,5.154940
...,...,...,...,...
227701,Vinegar Hill (Oprah's Book Club (Paperback)),265,2.245283,2.261879
233635,Whispers,286,2.199301,2.215805
143377,Presumed Innocent,294,2.139456,2.156913
94382,Isle of Dogs,288,2.000000,2.021122


In [21]:
popular_df=popular_df.merge(books,on='Book-Title').drop_duplicates('Book-Title')[['Book-Title','Book-Author','Image-URL-L','Number-of-Users-Rated','Average-Ratings','Popularity']]

In [22]:
popular_df['Image-URL-L'][0]

'http://images.amazon.com/images/P/0439136350.01.LZZZZZZZ.jpg'

## Collaborative Filtering Based Recommending System

In [23]:
# getting the values of people who have voted more than 100 times on books - knowledgeable_users
x = ratings_with_book_name.groupby('User-ID').count()['Book-Rating'] >150
# boolean indexing
knowledgeable_users = x[x].index

In [24]:
knowledgeable_users

Int64Index([   254,   1733,   1903,   2033,   2110,   2276,   2766,   2891,
              2977,   3363,
            ...
            274301, 274308, 274808, 275970, 276680, 277427, 277478, 277639,
            278188, 278418],
           dtype='int64', name='User-ID', length=1115)

In [25]:
# based on that filtering ratings_with_book_name dataframe
filtered_rating = ratings_with_book_name[ratings_with_book_name['User-ID'].isin(knowledgeable_users)]
filtered_rating

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L,User-ID,Book-Rating
3,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,11676,8
6,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,85526,0
7,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,96054,0
10,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,177458,0
21,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...,110912,10
...,...,...,...,...,...,...,...,...,...,...
1031127,1582380805,Tropical Rainforests: 230 Species in Full Colo...,"Allen M., Ph.D. Young",2001,Golden Guides from St. Martin's Press,http://images.amazon.com/images/P/1582380805.0...,http://images.amazon.com/images/P/1582380805.0...,http://images.amazon.com/images/P/1582380805.0...,275970,0
1031128,1845170423,Cocktail Classics,David Biggs,2004,Connaught,http://images.amazon.com/images/P/1845170423.0...,http://images.amazon.com/images/P/1845170423.0...,http://images.amazon.com/images/P/1845170423.0...,275970,7
1031133,006008667X,Lily Dale : The True Story of the Town that Ta...,Christine Wicker,2004,HarperSanFrancisco,http://images.amazon.com/images/P/006008667X.0...,http://images.amazon.com/images/P/006008667X.0...,http://images.amazon.com/images/P/006008667X.0...,276680,0
1031134,0192126040,Republic (World's Classics),Plato,1996,Oxford University Press,http://images.amazon.com/images/P/0192126040.0...,http://images.amazon.com/images/P/0192126040.0...,http://images.amazon.com/images/P/0192126040.0...,276680,0


In [26]:
# Based on book title how many books have more than 50 ratings-famous_books
y = filtered_rating.groupby('Book-Title').count()['Book-Rating']>=50
famous_books = y[y].index
famous_books

Index(['1984', '1st to Die: A Novel', '2nd Chance', '4 Blondes',
       '84 Charing Cross Road',
       'A Beautiful Mind: The Life of Mathematical Genius and Nobel Laureate John Nash',
       'A Bend in the Road', 'A Case of Need',
       'A Child Called \It\": One Child's Courage to Survive"',
       'A Civil Action',
       ...
       'Without Remorse', 'Wizard and Glass (The Dark Tower, Book 4)',
       'Women Who Run with the Wolves',
       'Word Freak: Heartbreak, Triumph, Genius, and Obsession in the World of Competitive Scrabble Players',
       'Wuthering Heights', 'Year of Wonders', 'You Belong To Me',
       'Zen and the Art of Motorcycle Maintenance: An Inquiry into Values',
       'Zoya', '\O\" Is for Outlaw"'],
      dtype='object', name='Book-Title', length=868)

In [27]:
# # Based on Author Name how many books have more than 50 ratings-famous_authors
# y = filtered_rating.groupby('Book-Author').count()['Book-Title']>=5
# famous_authors = y[y].index
# famous_authors

Index(['A A Milne', 'A Adams', 'A. A. Attanasio', 'A. A. Milne',
       'A. B. Yehoshua', 'A. Bates', 'A. C. Bhaktivedanta Swami Prabhupada',
       'A. C. Crispin', 'A. C. Doyle', 'A. C. H. Smith',
       ...
       'Zora Neale Hurston', 'Zsuzsanna E. Budapest', 'Zz Packer',
       '\Pickles\""', 'an Na', 'asha bandele', 'bell hooks',
       'jr., Richard Herman', 'stephen R Donaldson', 'tk'],
      dtype='object', name='Book-Author', length=13583)

In [28]:
final_ratings_book = filtered_rating[filtered_rating['Book-Title'].isin(famous_books)]

In [29]:
# final_ratings_author = filtered_rating[filtered_rating['Book-Author'].isin(famous_authors)]

In [30]:
# generate pivot_table based on above two filtering
pt = final_ratings_book.pivot_table(index='Book-Title',columns='User-ID',values='Book-Rating')

In [31]:
pt.fillna(0,inplace=True)
pt1.fillna(0,inplace=True)

NameError: ignored

In [ ]:
pt

In [ ]:
pt1

In [ ]:
similarity_scores_books = cosine_similarity(pt)
similarity_scores_author = cosine_similarity(pt1)

In [314]:
similarity_scores_books.shape

(868, 868)

In [315]:
similarity_scores_author.shape

(1659, 1659)

In [316]:
def recommend_book(book_name):
    # index fetch
    index = np.where(pt.index==book_name)[0][0]
    # aftre enumerate we get item index as well with respect to 
    similar_items = sorted(list(enumerate(similarity_scores_books[index])),key=lambda x:x[1],reverse=True)[1:5]
    
    data = []
    for i in similar_items:
        item = []
        temp_df = books[books['Book-Title'] == pt.index[i[0]]]
        item.extend(list(temp_df.drop_duplicates('Book-Title')['Book-Title'].values))
        item.extend(list(temp_df.drop_duplicates('Book-Title')['Book-Author'].values))
        item.extend(list(temp_df.drop_duplicates('Book-Title')['Image-URL-L'].values))
        
        data.append(item)
    
    return data

In [320]:
def recommend_author(book_author):
    # index fetch
    index = np.where(pt.index==book_author)[0][0]
    # aftre enumerate we get item index as well with respect to 
    similar_items = sorted(list(enumerate(similarity_scores_author[index])),key=lambda x:x[1],reverse=True)[1:5]
    
    data = []
    for i in similar_items:
        item = []
        temp_df = books[books['Book-Author'] == pt.index[i[0]]]
        item.extend(list(temp_df.drop_duplicates('Book-Title')['Book-Title'].values))
        item.extend(list(temp_df.drop_duplicates('Book-Title')['Book-Author'].values))
        item.extend(list(temp_df.drop_duplicates('Book-Title')['Image-URL-L'].values))
        
        data.append(item)
    
    return data

In [321]:
recommend_book("Harry Potter and the Prisoner of Azkaban (Book 3)")

[['Harry Potter and the Goblet of Fire (Book 4)',
  'J. K. Rowling',
  'http://images.amazon.com/images/P/0439139597.01.LZZZZZZZ.jpg'],
 ['Harry Potter and the Chamber of Secrets (Book 2)',
  'J. K. Rowling',
  'http://images.amazon.com/images/P/0439064872.01.LZZZZZZZ.jpg'],
 ["Harry Potter and the Sorcerer's Stone (Book 1)",
  'J. K. Rowling',
  'http://images.amazon.com/images/P/0590353403.01.LZZZZZZZ.jpg'],
 ['Harry Potter and the Order of the Phoenix (Book 5)',
  'J. K. Rowling',
  'http://images.amazon.com/images/P/043935806X.01.LZZZZZZZ.jpg']]

In [322]:
recommend_author("J. K. Rowling")

IndexError: ignored